# NCAAM match predictions

## Imports

In [1]:
import os
import re
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

## Import data

In [68]:
path_datasets = 'data/MDataFiles_Stage1/'

df_regular_compact = pd.read_csv(path_datasets + 'MRegularSeasonCompactResults.csv')
df_regular_detailed = pd.read_csv(path_datasets + 'MRegularSeasonDetailedResults.csv')
df_teams = pd.read_csv(path_datasets + 'MTeams.csv')
df_seeds = pd.read_csv(path_datasets + 'MNCAATourneySeeds.csv')
coaches = pd.read_csv(path_datasets + 'MTeamCoaches.csv')
df_tourney_compact = pd.read_csv(path_datasets + 'MNCAATourneyCompactResults.csv')
df_tourney_detailed = pd.read_csv(path_datasets + 'MNCAATourneyDetailedResults.csv')

In [74]:
df_seeds.groupby(by="TeamID").count()

,Season,Seed
TeamID,,
1101,1,1
1102,2,2
1103,4,4
1104,16,16
1105,1,1
1106,4,4
1107,5,5
1108,2,2
1110,3,3


In [3]:
df_regular_compact.describe()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,NumOT
count,161552.000000,161552.000000,161552.000000,161552.000000,161552.000000,161552.000000,161552.000000
mean,2003.228731,74.665359,1286.919004,76.795125,1282.637498,64.692526,0.045137
std,10.058660,33.670352,104.521630,12.066260,104.742961,11.349610,0.249751
min,1985.000000,0.000000,1101.000000,34.000000,1101.000000,20.000000,0.000000
25%,1995.000000,47.000000,1198.000000,69.000000,1191.000000,57.000000,0.000000
50%,2004.000000,77.000000,1284.000000,76.000000,1280.000000,64.000000,0.000000
75%,2012.000000,103.000000,1380.000000,84.000000,1375.000000,72.000000,0.000000
max,2019.000000,132.000000,1466.000000,186.000000,1466.000000,150.000000,6.000000


### Check data sanity

In [4]:
print('Regular Season compact Results :\n', df_regular_compact.isnull().sum().T)
print("-"*10)

print('Regular Season detailed Results :\n', df_regular_detailed.isnull().sum().T)
print("-"*10)

print('Teamsts :\n', df_teams.isnull().sum().T)
print("-"*10)

print('Seeds :\n', df_seeds.isnull().sum().T)
print("-"*10)

print('Coaches :\n', coaches.isnull().sum().T)
print("-"*10)

print('Tourney compact Results :\n', df_tourney_compact.isnull().sum().T)
print("-"*10)

print('Tourney compact Results :\n', df_tourney_detailed .isnull().sum().T)
print("-"*10)

Regular Season compact Results :
 Season     0
DayNum     0
WTeamID    0
WScore     0
LTeamID    0
LScore     0
WLoc       0
NumOT      0
dtype: int64
----------
Regular Season detailed Results :
 Season     0
DayNum     0
WTeamID    0
WScore     0
LTeamID    0
LScore     0
WLoc       0
NumOT      0
WFGM       0
WFGA       0
WFGM3      0
WFGA3      0
WFTM       0
WFTA       0
WOR        0
WDR        0
WAst       0
WTO        0
WStl       0
WBlk       0
WPF        0
LFGM       0
LFGA       0
LFGM3      0
LFGA3      0
LFTM       0
LFTA       0
LOR        0
LDR        0
LAst       0
LTO        0
LStl       0
LBlk       0
LPF        0
dtype: int64
----------
Teamsts :
 TeamID           0
TeamName         0
FirstD1Season    0
LastD1Season     0
dtype: int64
----------
Seeds :
 Season    0
Seed      0
TeamID    0
dtype: int64
----------
Coaches :
 Season         0
TeamID         0
FirstDayNum    0
LastDayNum     0
CoachName      0
dtype: int64
----------
Tourney compact Results :
 Season    

### Concat match details in one dataframe

In [5]:
# add match type
df_regular_detailed["Type"] = "regular"
df_tourney_detailed["Type"] = "tourney"

# We join the data from the regular matchs and tourney matchs
df_match_detailed = pd.concat([df_regular_detailed, df_tourney_detailed])

# We create an idea for a match between two teams : team1_team2(team1 < team2)
df_match_detailed["Match"] = df_match_detailed \
                                .apply(lambda row: "_".join(map(str, sorted([row["WTeamID"], row["LTeamID"]]))), axis=1)

df_match_detailed["Team1"] = df_match_detailed["Match"].apply(lambda x: int(x.split("_")[0]))
df_match_detailed["Team2"] = df_match_detailed["Match"].apply(lambda x: int(x.split("_")[1]))

df_match_detailed["Label"] = df_match_detailed.apply(lambda row: 1 if row["WTeamID"] == row["Team1"] else 0, axis=1)

# WLoc to numeric type
df_match_detailed["WLoc"] = df_match_detailed["WLoc"].map({"H": 0, "A": 1, "N": 3})
# Type to numeric type
df_match_detailed["Type"] = df_match_detailed["Type"].map({"regular": 0, "tourney": 1})

## Create interesting metrics

Features initiales Luc:

* % de victoires
* % de victoires à domicile et à l'exterieur
* Nombre de points marqués et encaissés en moyenne
* Nombre d'interceptions et de rebonds
* % de réussites à 3pts
* Nombre de lancers francs tentés/marqués

Features à ajouter possiblement:

* L'équipe joue à domicile ?
* % de victoire contre une équipe de rang équivalent (à 1 ou deux seed près?)

Le travail se répartie entre les différentes personnes du groupe (je prend les trois dernières)

### Nombre d'interceptions et de rebonds par match et par saison

Pour créer ces paramètres nous avons besoin des dataframes suivant :

Comptés commme une interception : blocks, steals

Comptés comme rebond : offensive_rebounds, defensive_rebounds

In [6]:
df_match_detailed.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF',
       'Type', 'Match', 'Team1', 'Team2', 'Label'],
      dtype='object')

#### Nombre de rebond en moyenne par match de chaque équipe sur chaque saison

In [7]:
# studied_parameter = "DR"
# data = df_match_detailed[["Season", "WTeamID", "LTeamID", "WDR" , "LDR"]].values
# winners_rebound_count = np.zeros(data.shape[0])
# losers_rebound_count = np.zeros(data.shape[0])
# for i, match in enumerate(data):
#     season = match[0]
#     winner = match[1]
#     loser = match[2]
    
#     previous_seasons = data[data[:,0] < season]
#     winner_previous_wins = previous_seasons[previous_seasons[:,1] == winner]
#     winner_previous_loses = previous_seasons[previous_seasons[:,2] == winner]
#     loser_previous_wins = previous_seasons[previous_seasons[:,1] == loser]
#     loser_previous_loses = previous_seasons[previous_seasons[:,2] == loser]
    
#     winner_rebound_count = np.sum(winner_previous_wins) + np.sum(winner_previous_loses)
#     loser_rebound_count = np.sum(loser_previous_wins) + np.sum(winner_previous_loses)
    
#     winners_rebound_count[i] = winner_rebound_count
#     losers_rebound_count[i] = loser_rebound_count

In [8]:
df_tmp = df_match_detailed[["Season", "WTeamID", "LTeamID", "WDR" , "LDR", "WOR", 'LOR']]
df_rebound_count_metric = pd.DataFrame(columns=["Season", "TeamID", "rebound_count"])
for season in df_tmp.Season.unique():
    df_season = df_tmp.loc[df_tmp.Season == season]
    for team in df_teams.TeamID.unique():
        rebound_count_win = df_season[["WDR", "WOR"]].loc[df_season.WTeamID == team].mean().sum()
        rebound_count_lose = df_season[["LDR", "LOR"]].loc[df_season.LTeamID == team].mean().sum()
        new_line = pd.DataFrame({"Season":[season], 
                                 "TeamID":[team], 
                                 "rebound_count":[rebound_count_win + rebound_count_lose]})                                                                 
        df_rebound_count_metric = df_rebound_count_metric.append(new_line, ignore_index=True)

In [9]:
df_rebound_count_metric.head()

,Season,TeamID,rebound_count
0,2003,1101,0.000000
1,2003,1102,42.541667
2,2003,1103,59.489011
3,2003,1104,73.274510
4,2003,1105,75.639098


#### Nombre de d'intercéptions en moyenne par match de chaque équipe sur chaque saison

In [10]:
df_tmp = df_match_detailed[["Season", "WTeamID", "LTeamID", "WBlk" , "LBlk", "WStl", 'LStl']]
df_intercept_count_metric = pd.DataFrame(columns=["Season", "TeamID", "intercept_count"])
for season in df_tmp.Season.unique():
    df_season = df_tmp.loc[df_tmp.Season == season]
    for team in df_teams.TeamID.unique():
        intercept_count_win = df_season[["WBlk", "WStl"]].loc[df_season.WTeamID == team].mean().sum()
        intercept_count_lose = df_season[["LBlk", "LStl"]].loc[df_season.LTeamID == team].mean().sum()
        new_line = pd.DataFrame({"Season":[season], 
                                 "TeamID":[team], 
                                 "intercept_count":[intercept_count_win + intercept_count_lose]})                                                                 
        df_intercept_count_metric = df_intercept_count_metric.append(new_line, ignore_index=True)

In [11]:
df_intercept_count_metric.shape

(6239, 3)

### Pourcentage de réussites à 3 points par match et par saison

In [12]:
df_tmp = df_match_detailed[["Season", "WTeamID", "LTeamID", "WFGM3" , "LFGM3", "WFGA3", 'LFGA3']]
df_3_points_rate_metric = pd.DataFrame(columns=["Season", "TeamID", "3_points_rate"])
for season in df_tmp.Season.unique():
    df_season = df_tmp.loc[df_tmp.Season == season]
    for team in df_teams.TeamID.unique():
        
        three_points_made_count_win = df_season["WFGM3"].loc[df_season.WTeamID == team].mean()
        three_points_made_count_lose = df_season["LFGM3"].loc[df_season.LTeamID == team].mean()
        
        three_points_attempted_count_win = df_season["WFGA3"].loc[df_season.WTeamID == team].mean()
        three_points_attempted_count_lose = df_season["LFGA3"].loc[df_season.LTeamID == team].mean()
        
        three_points_rate = (three_points_made_count_win + three_points_made_count_lose)/ (three_points_attempted_count_win + three_points_attempted_count_lose)
        new_line = pd.DataFrame({"Season":[season], 
                                 "TeamID":[team], 
                                 "3_points_rate":[three_points_rate]})                                                                 
        df_3_points_rate_metric = df_3_points_rate_metric.append(new_line, ignore_index=True)

In [13]:
df_3_points_rate_metric.head()

,Season,TeamID,3_points_rate
0,2003,1101,NaN
1,2003,1102,0.386952
2,2003,1103,0.340024
3,2003,1104,0.318412
4,2003,1105,0.379061


### Pourcentage de réussites au lancé franc par match et par saison

In [14]:
df_tmp = df_match_detailed[["Season", "WTeamID", "LTeamID", "WFTM" , "LFTM", "WFTA", 'LFTA']]
df_free_throw_rate_metric = pd.DataFrame(columns=["Season", "TeamID", "free_throw_rate"])
for season in df_tmp.Season.unique():
    df_season = df_tmp.loc[df_tmp.Season == season]
    for team in df_teams.TeamID.unique():
        
        free_throw_made_count_win = df_season["WFTM"].loc[df_season.WTeamID == team].mean()
        free_throw_made_count_lose = df_season["LFTM"].loc[df_season.LTeamID == team].mean()
        
        free_throw_attempted_count_win = df_season["WFTA"].loc[df_season.WTeamID == team].mean()
        free_throw_attempted_count_lose = df_season["LFTA"].loc[df_season.LTeamID == team].mean()
        
        free_throw_rate = (free_throw_made_count_win + free_throw_made_count_lose)/ (free_throw_attempted_count_win + free_throw_attempted_count_lose)
        new_line = pd.DataFrame({"Season":[season], 
                                 "TeamID":[team], 
                                 "free_throw_rate":[free_throw_rate]})                                                                 
        df_free_throw_rate_metric = df_free_throw_rate_metric.append(new_line, ignore_index=True)

### Merge all metrics in one DataFrame

In [15]:
df_metrics = df_rebound_count_metric
metrics = [df_intercept_count_metric, df_3_points_rate_metric, df_free_throw_rate_metric]
for metric in metrics:
    df_metrics = pd.merge(df_metrics, metric)

In [16]:
df_metrics

,Season,TeamID,rebound_count,intercept_count,3_points_rate,free_throw_rate
0,2003,1101,0.000000,0.000000,NaN,NaN
1,2003,1102,42.541667,16.104167,0.386952,0.651839
2,2003,1103,59.489011,19.153846,0.340024,0.735753
3,2003,1104,73.274510,20.161765,0.318412,0.713187
4,2003,1105,75.639098,23.969925,0.379061,0.719524
5,2003,1106,72.902564,23.220513,0.347425,0.644421
6,2003,1107,60.476190,17.761905,0.365027,0.748752
7,2003,1108,72.710526,23.906015,0.327446,0.673882
8,2003,1109,0.000000,0.000000,NaN,NaN
9,2003,1110,66.178571,16.151786,0.374904,0.682747


## Metrics to features

In [82]:
def mean_year_offset_metric(df_metrics, offset):
    df_offset = pd.DataFrame()
    for season in df_metrics.Season.unique():
        df_offset_season = df_metrics.loc[(df_metrics.Season < season)& (df_metrics.Season > season-offset)].groupby(by="TeamID").mean()
        df_offset_season["Season"] = season
        df_offset = pd.concat([df_offset, df_offset_season], join="outer")
    return df_offset.reset_index()

In [83]:
df_offset = mean_year_offset_metric(df_metrics, offset)
df_offset

,TeamID,rebound_count,intercept_count,3_points_rate,free_throw_rate,Season
0,1101,0.000000,0.000000,NaN,NaN,2004
1,1102,42.541667,16.104167,0.386952,0.651839,2004
2,1103,59.489011,19.153846,0.340024,0.735753,2004
3,1104,73.274510,20.161765,0.318412,0.713187,2004
4,1105,75.639098,23.969925,0.379061,0.719524,2004
5,1106,72.902564,23.220513,0.347425,0.644421,2004
6,1107,60.476190,17.761905,0.365027,0.748752,2004
7,1108,72.710526,23.906015,0.327446,0.673882,2004
8,1109,0.000000,0.000000,NaN,NaN,2004
9,1110,66.178571,16.151786,0.374904,0.682747,2004


In [143]:
df_tmp = df_match_detailed[["Season", "Team1", "Team2"]]

mets = df_offset.columns[df_offset.columns != "Season"]
mets = mets[mets != "TeamID"]

df_metrics_1 = df_offset.rename(columns={"TeamID":"Team1"})
df_metrics_2 = df_offset.rename(columns={"TeamID":"Team2"})
df_metrics_1 = df_offset.rename(columns={metric:"Team1_{}".format(metric) for metric in mets})
df_metrics_2 = df_offset.rename(columns={metric:"Team2_{}".format(metric) for metric in mets})

df_metrics_1 = df_metrics_1.rename(columns={"TeamID":"Team1"})
df_metrics_2 = df_metrics_2.rename(columns={"TeamID":"Team2"})

df_tmp = pd.merge(left=df_tmp, right=df_metrics_1, how='left', on=['Season', 'Team1'])
df_matches = pd.merge(left=df_tmp, right=df_metrics_2, on=['Season', 'Team2'])

In [144]:
df_matches

,Season,Team1,Team2,Team1_rebound_count,Team1_intercept_count,Team1_3_points_rate,Team1_free_throw_rate,Team2_rebound_count,Team2_intercept_count,Team2_3_points_rate,Team2_free_throw_rate
0,2004,1266,1385,73.259259,17.629630,0.373151,0.789614,75.629808,28.024038,0.313264,0.661278
1,2004,1193,1385,72.794258,25.282297,0.326380,0.719717,75.629808,28.024038,0.313264,0.661278
2,2004,1220,1385,82.714286,17.708333,0.332156,0.594527,75.629808,28.024038,0.313264,0.661278
3,2004,1181,1385,71.335165,25.153846,0.338082,0.693138,75.629808,28.024038,0.313264,0.661278
4,2004,1383,1385,67.482143,25.857143,0.360137,0.694078,75.629808,28.024038,0.313264,0.661278
5,2004,1210,1385,77.571429,26.857143,0.347162,0.677365,75.629808,28.024038,0.313264,0.661278
6,2004,1335,1385,62.303030,17.863636,0.380634,0.719588,75.629808,28.024038,0.313264,0.661278
7,2004,1221,1385,69.480000,23.600000,0.322540,0.690943,75.629808,28.024038,0.313264,0.661278
8,2004,1310,1385,72.995098,21.877451,0.322898,0.687687,75.629808,28.024038,0.313264,0.661278
9,2004,1371,1385,71.607843,20.686275,0.305501,0.687282,75.629808,28.024038,0.313264,0.661278
